# Experimentação de Cenários utilizando o GEM

### Importação de dependências

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gem
from gem.tools.python_code_tool import PythonCodeTool
from gem.tools.tool_env_wrapper import ToolEnvWrapper
from gem.wrappers.wrapper_factory import WRAPPER_FACTORY

/home/micael/mestrado/inf2070-drl-gem-study/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Implementação da ação do agente LLM

In [2]:
# --- Função Auxiliar para Geração do LLM ---
def generate_action(prompt, model, tokenizer, device, max_new_tokens=1024):
    """
    Gera uma ação (texto) a partir de um prompt (observação) usando o LLM.
    """
    # O 'obs' do concat_chat já é o prompt formatado.
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Garantir que pad_token_id está definido para geração open-ended
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    # Gerar a resposta
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # Decodificar apenas os tokens *novos* gerados, ignorando o prompt de entrada
    input_length = inputs["input_ids"].shape[1]
    new_tokens = outputs[0, input_length:]
    action_text = tokenizer.decode(new_tokens, skip_special_tokens=True)
    
    return action_text

### Ciclo de Interação Agente-Ambiente

In [3]:
base_env = gem.make("math:GSM8K")
# Esta é a ferramenta que o LLM pode decidir usar
tool = PythonCodeTool()
# Este wrapper dá ao ambiente a capacidade de entender e executar a ferramenta
tool_env = ToolEnvWrapper(base_env, tools=[tool])

device = "cpu"

# Lista de modelos para testar
models = [
    # "microsoft/Phi-3-mini-4k-instruct", # Usando Phi-3 no lugar do Phi-4 que não existe
    # "Qwen/Qwen2-0.5B-Instruct",      # Usando Qwen2 0.5B no lugar do Qwen3 0.6B
    "deepseek-ai/deepseek-coder-1.3b-instruct" # Usando um modelo de código Deepseek
]

# --- 2. Loop Principal (Iterar sobre cada modelo) ---
for model_name in models:
    print(f"\n{'='*20} Using model: {model_name} {'='*20}")
    
    try:
        # --- 3. Carregar Modelo e Tokenizer ---
        # trust_remote_code=True é frequentemente necessário para modelos mais novos
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        llm_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32 # Otimização para GPU
        ).to(device)
        llm_model.eval() # Definir modelo para modo de avaliação

        # --- 4. Aplicar o Wrapper de Chat ---
        # Este wrapper formata o histórico de observações em um prompt de chat
        # que o LLM entende, incluindo a descrição das ferramentas.
        chat_env = WRAPPER_FACTORY["concat_chat"](
            tool_env, tokenizer=tokenizer
        )

        # --- 5. Iniciar o Episódio ---
        obs, info = chat_env.reset()
        print(f"[Initial Observation (Problem)]:\n{obs}")
        print(f"[Initial Info]: {info}")
        print("-" * 30)

        episode_reward = 0
        step_count = 0
        
        # --- 6. Loop de Interação (Agente-Ambiente) ---
        while True:
            step_count += 1
            print(f"\n[Step {step_count}]")

            # === AÇÃO DO LLM ===
            # Em vez de uma ação aleatória, geramos uma ação com o LLM
            with torch.no_grad(): # Desativar cálculo de gradiente para inferência
                action = generate_action(obs, llm_model, tokenizer, device)
            
            # --- LOG: Ação do LLM ---
            print(f"\n[LLM Action]:\n{action}\n")
            
            # === PASSO NO AMBIENTE ===
            # O 'chat_env' recebe a ação de texto.
            # O 'ToolEnvWrapper' (dentro dele) irá procurar por <python>...</python>
            # Se encontrar, executa o código e retorna o stdout/stderr.
            # Se não encontrar, passa a ação direto para o 'base_env'.
            next_observation, reward, terminated, truncated, info = chat_env.step(action)
            
            # --- LOG: Feedback do Ambiente ---
            print(f"[Environment Feedback (Next Observation)]:\n{next_observation}")
            print(f"[Reward]: {reward}")
            print(f"[Info]: {info}")
            print("-" * 30)

            # Atualizar a observação para o próximo loop
            obs = next_observation
            episode_reward += reward

            # Condição de parada
            if terminated or truncated:
                print(f"\n{'='*10} Episode Finished {'='*10}")
                print(f"Total Reward: {episode_reward}")
                print(f"Final Info: {info}")
                break
            
            # Limite de segurança (opcional, mas bom para debug)
            if step_count > 10:
                print("Step limit reached, truncating episode.")
                break

        chat_env.close()
        print(f"=== Finished using model: {model_name} ===")

    except Exception as e:
        print(f"Failed to run model {model_name}. Error: {e}")
        # Limpar memória da GPU se houver erro antes de ir para o próximo modelo
        if 'llm_model' in locals():
            del llm_model
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

print("\nAll models processed.")


==================== Using model: deepseek-ai/deepseek-coder-1.3b-instruct ====================


`torch_dtype` is deprecated! Use `dtype` instead!


[Initial Observation (Problem)]:
<｜begin▁of▁sentence｜>You are an AI programming assistant, utilizing the Deepseek Coder model, developed by Deepseek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer
### Instruction:
Three siblings are born 5 years apart, each. If the eldest child is 20 years old now, what's the total of the ages of the three siblings 10 years from now?
Solve the above problem step by step. You now have the ability to selectively write executable Python code to enhance your reasoning process. The Python code will be executed by an external sandbox, and the output (after "Code execution result: ") is returned to aid your reasoning and help you arrive at the final answer. The Python code should be complete scripts, including necessary imports, wrapped within <python>...</python> tags or using ```python...``` code block. Re

### Ciclo Interação Agente-Ambiente (sem ferramentas)

In [8]:
# --- 1. Configuração do Ambiente Base (Sem Ferramentas) ---
# Este é o ambiente principal com o problema (ex: GSM8K)
base_env = gem.make("game:GuessTheNumber-v0-easy")
# Ferramentas removidas desta versão
device = "cpu"
# Lista de modelos para testar
models = [
    # "microsoft/Phi-3-mini-4k-instruct", # Usando Phi-3 no lugar do Phi-4 que não existe
    # "Qwen/Qwen2-0.5B-Instruct",      # Usando Qwen2 0.5B no lugar do Qwen3 0.6B
    # "deepseek-ai/DeepSeek-R1",
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0" # Usando um modelo de código Deepseek
]

# --- 2. Loop Principal (Iterar sobre cada modelo) ---
for model_name in models:
    print(f"\n{'='*20} Using model: {model_name} {'='*20}")
    
    try:
        # --- 3. Carregar Modelo e Tokenizer ---
        # trust_remote_code=True é frequentemente necessário para modelos mais novos
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        llm_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float32 # Usar float32 para CPU
        ).to(device)
        llm_model.eval() # Definir modelo para modo de avaliação

        # --- 4. Aplicar o Wrapper de Chat ---
        # Este wrapper formata o histórico de observações em um prompt de chat
        # que o LLM entende.
        # Alterado: 'tool_env' foi substituído por 'base_env'
        chat_env = WRAPPER_FACTORY["concat_chat"](
            base_env, tokenizer=tokenizer
        )

        # --- 5. Iniciar o Episódio ---
        obs, info = chat_env.reset()
        print(f"[Initial Observation (Problem)]:\n{obs}")
        print(f"[Initial Info]: {info}")
        print("-" * 30)

        episode_reward = 0
        step_count = 0
        
        # --- 6. Loop de Interação (Agente-Ambiente) ---
        while True:
            step_count += 1
            print(f"\n[Step {step_count}]")

            # === AÇÃO DO LLM ===
            # Em vez de uma ação aleatória, geramos uma ação com o LLM
            with torch.no_grad(): # Desativar cálculo de gradiente para inferência
                action = generate_action(obs, llm_model, tokenizer, device)
            
            # --- LOG: Ação do LLM ---
            print(f"\n[LLM Action]:\n{action}\n")
            
            # === PASSO NO AMBIENTE ===
            # O 'chat_env' recebe a ação de texto.
            # Como não há ToolEnvWrapper, a ação será passada
            # diretamente para o 'base_env' como a resposta final.
            next_observation, reward, terminated, truncated, info = chat_env.step(action)
            
            # --- LOG: Feedback do Ambiente ---
            print(f"[Environment Feedback (Next Observation)]:\n{next_observation}")
            print(f"[Reward]: {reward}")
            print(f"[Info]: {info}")
            print("-" * 30)

            # Atualizar a observação para o próximo loop
            obs = next_observation
            episode_reward += reward

            # Condição de parada
            if terminated or truncated:
                print(f"\n{'='*10} Episode Finished {'='*10}")
                print(f"Total Reward: {episode_reward}")
                print(f"Final Info: {info}")
                break
            
            # Limite de segurança (opcional, mas bom para debug)
            if step_count > 10:
                print("Step limit reached, truncating episode.")
                break

        chat_env.close()
        print(f"=== Finished using model: {model_name} ===")

    except Exception as e:
        print(f"Failed to run model {model_name}. Error: {e}")
        # Limpar memória da GPU se houver erro antes de ir para o próximo modelo
        if 'llm_model' in locals():
            del llm_model
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

print("\nAll models processed.")


==================== Using model: TinyLlama/TinyLlama-1.1B-Chat-v1.0 ====================
[Initial Observation (Problem)]:
<|user|>
You are playing Guess The Number.
You have to guess the number between 1 and 10 (inclusive) within 4 turns.
As you enter your guess, the game will provide you with hints such as the target number is 'higher' or 'lower'.
You may provide your response in any manner. Only the number that is wrapped inside \boxed{} will be considered as your guess, for example, \boxed{10}.
As you play, the history of your guesses will be appended below. Use the information to complete the game before you run out of guesses.
Enter your first guess to start the game.
</s>
<|assistant|>
Enter your next guess.
[Initial Info]: {'suffix': 'Enter your next guess.'}
------------------------------

[Step 1]

[LLM Action]:


Enter your next guess.

Enter your next guess.

Enter your next guess.

Enter your next guess.

Enter your next guess.

Enter your next guess.

Enter your next gue